In [ ]:
import os
import argparse
import json
import csv

import pandas as pd

# please run command "pip install pyvis" in terminal if pyvis not installed
from pyvis.network import Network

In [ ]:
def extract_network_file(network_file):
    json_file = open(network_file)
    network = json.load(json_file)
    json_file.close()

    return network

In [ ]:
# Pass your mnms json file here in parameter
mnms_network = extract_network_file("lyon_mnms_restricted_gtfs_bus_tram_metro.json")
#df_demand = extract_demand_file("fichier_demandes_full.csv")

roads = mnms_network.get("ROADS")
layers = mnms_network.get("LAYERS")

nodes = roads.get("NODES")
sections = roads.get("SECTIONS")
stops = roads.get("STOPS")

In [ ]:
net = Network(height="800px", width="100%", notebook=True, filter_menu=True, cdn_resources="remote")

In [ ]:
for id, node in nodes.items():
    id_node = node["id"]
    if id_node not in stops:
        x_node = float(node["position"][0])
        y_node = -float(node["position"][1])
        net.add_node(id_node, label=' ', title=id_node, x=x_node, y=y_node, size=2, color="grey", group="CAR")

In [ ]:
for layer in layers:
    if layer["VEH_TYPE"] == "mnms.vehicles.veh_type.Bus":
        lines = layer["LINES"]
        for line in lines:
            line_stops = line["STOPS"]   
            lsections = line["SECTIONS"]
            for line_stop in line_stops:
                stop = stops[line_stop]
                id_stop = stop["id"]
                x_stop = float(stop["absolute_position"][0])
                y_stop = -float(stop["absolute_position"][1])
                net.add_node(id_stop, label=' ', title=id_stop, x=x_stop, y=y_stop, size=2, color="grey", group="BUS")
            for lsection in lsections:
                for isection in lsection:
                    section = sections[isection]               
                    up_node = section["upstream"]
                    down_node = section["downstream"]
                    if str(isection).startswith("BUS"):
                        net.add_edge(up_node, down_node, title=isection, width=1, color="red")
                    else:
                        net.add_edge(up_node, down_node, title=isection, width=1, color="green")

In [ ]:
for id, section in sections.items():
    id_section = section["id"]
    upnode = section["upstream"]
    downnode = section["downstream"]
    if upnode not in stops and downnode not in stops:
        net.add_edge(upnode, downnode, title=id_section, width=1, color="grey")

In [ ]:
net.toggle_physics(False)
net.toggle_drag_nodes(False)
net.show("lyon_restricted_mapmatching_gtfs.html")